<a href="https://colab.research.google.com/github/dokdin/Portfolio3/blob/master/LG_AI_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 파일 디렉토리 
PATH='/content/drive/MyDrive/Data/LG/'
CSV_PATH='/content/drive/MyDrive/Data/LG/CSV/'

In [ ]:
!pip install pycaret

In [ ]:
# 필요 라이브러리 호출 

from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

# 필요 함수 정의

def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    # hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year,month, day)

In [ ]:
# 파일 불러오기 
train_err = pd.read_csv(PATH+"train_err_data.csv")
train_quality = pd.read_csv(PATH+'train_quality_data.csv')
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
test_err = pd.read_csv(PATH+'test_err_data.csv')

# train data

In [ ]:
# 시간 함수 적용하기 
# train_err['datetime']=train_err['time'].apply(make_datetime)
train_err['datetime']=train_err['time'].apply(make_datetime)

In [ ]:
print(train_err['datetime'].min())
print(train_err['datetime'].max())

In [ ]:
# 시간변경
filtered_df=train_err.loc[train_err["datetime"].between('2020-10-31','2020-11-30')]

In [ ]:
# 시간데이터 수정 후 
print(str(train_err.shape))
print(str(filtered_df.shape))

In [ ]:
# datetime에서 정수로 변환
filtered_df['datetime'] = filtered_df['datetime'].dt.strftime('%Y-%m-%d')

In [ ]:
def test1(x):
    if x=='2020-11-01': return 1
    elif x=='2020-11-02': return 2
    elif x=='2020-11-03': return 3
    elif x=='2020-11-04': return 4
    elif x=='2020-11-06': return 5
    elif x=='2020-11-07': return 6
    elif x=='2020-11-08': return 7
    elif x=='2020-11-09': return 8
    elif x=='2020-11-10': return 9
    elif x=='2020-11-11': return 10
    elif x=='2020-11-12': return 11
    elif x=='2020-11-13': return 12
    elif x=='2020-11-14': return 13
    elif x=='2020-11-15': return 14
    elif x=='2020-11-16': return 15
    elif x=='2020-11-17': return 16
    elif x=='2020-11-18': return 17
    elif x=='2020-11-19': return 18
    elif x=='2020-11-20': return 19
    elif x=='2020-11-21': return 20
    elif x=='2020-11-22': return 21
    elif x=='2020-11-23': return 22
    elif x=='2020-11-24': return 23
    elif x=='2020-11-25': return 24
    elif x=='2020-11-26': return 25
    elif x=='2020-11-27': return 26
    elif x=='2020-11-28': return 27
    elif x=='2020-11-29': return 28
    elif x=='2020-11-30': return 29
    else: return 0

In [ ]:
filtered_df['datetime']=filtered_df['datetime'].apply(test1)

In [ ]:
# 일자 별 독립변수 생성 
id_error = filtered_df[['user_id','datetime']].values
errorr = np.zeros((15000,30))
for person_idx, err in tqdm(id_error):
    # person_idx - 10000 위치에 person_idx, errtype에 해당하는 error값을 +1
    errorr[person_idx - 10000,err - 1] += 1

In [ ]:
# errtype별 독립변수 생성 
id_errorr = filtered_df[['user_id','errtype']].values
error = np.zeros((15000,42))

for person_idx, err in tqdm(id_errorr):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - 10000,err - 1] += 1
error.shape

In [ ]:
# 데이터 프레임 화 
train = pd.DataFrame(data=error)
train2=pd.DataFrame(data=errorr)

In [ ]:
train=train.merge(train2,left_index=True, right_index=True )

In [ ]:
# 회원별 모델
train_err_model=filtered_df[['user_id','model_nm']].drop_duplicates()
model_list=np.sort(train_err_model['model_nm'].unique())
Model = pd.DataFrame()

for i in tqdm(range(len(model_list))):
  model=np.zeros((15000))
  train_err_model1=train_err_model[train_err_model['model_nm']==model_list[i]].reset_index(drop=True)
  model[train_err_model1.user_id.unique()-10000]=1
  Model[model_list[i]]=model

In [ ]:
# 확인
train=train.merge(Model,left_index=True, right_index=True )

In [ ]:
# 변수추가를 위한 종속변수 제거 
# train.drop(['problem'],axis='columns')

In [ ]:
Number_model = pd.DataFrame(columns=['user_id', 'Number_of_model'])
user_list=filtered_df.user_id.unique().tolist()
for i in tqdm(range(len(user_list))):
  count=len(filtered_df.loc[filtered_df.user_id==user_list[i]].model_nm.unique())
  Number_model=Number_model.append(pd.DataFrame([[user_list[i],count]],columns=['user_id','Number_of_model']),ignore_index=True)

In [ ]:
Number_model=Number_model.drop(['user_id'],axis='columns')

In [ ]:
# 확인
train=train.merge(Number_model,left_index=True, right_index=True )

In [ ]:
# 종속 변수
problem = np.zeros(15000)
problem[train_prob.user_id.unique()-10000] = 1 

train['problem'] = problem
# del error, problem

In [ ]:
train

In [ ]:
# AUTOML 실행
clf = setup(data = train, target = 'problem')

In [ ]:
best_5  = compare_models(sort = 'AUC', n_select = 5)

In [ ]:
blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft')

In [ ]:
pred_holdout = predict_model(blended)

In [ ]:
final_model = finalize_model(blended)

In [ ]:
final_model

# test data

In [ ]:
test_err = pd.read_csv(PATH+'test_err_data.csv')

In [ ]:
# 시간 함수 적용하기 
test_err['datetime']=test_err['time'].apply(make_datetime)

In [ ]:
# datetime에서 str로 변환
test_err['datetime'] = test_err['datetime'].dt.strftime('%Y-%m-%d')

In [ ]:
test_err=test_err.loc[test_err["datetime"].between('2020-10-31','2020-11-30')]
test_err=test_err.drop(['time'],axis='columns')

In [ ]:
test_err.info()

In [ ]:
def test1(x):
    if x=='2020-11-01': return 1
    elif x=='2020-11-02': return 2
    elif x=='2020-11-03': return 3
    elif x=='2020-11-04': return 4
    elif x=='2020-11-06': return 5
    elif x=='2020-11-07': return 6
    elif x=='2020-11-08': return 7
    elif x=='2020-11-09': return 8
    elif x=='2020-11-10': return 9
    elif x=='2020-11-11': return 10
    elif x=='2020-11-12': return 11
    elif x=='2020-11-13': return 12
    elif x=='2020-11-14': return 13
    elif x=='2020-11-15': return 14
    elif x=='2020-11-16': return 15
    elif x=='2020-11-17': return 16
    elif x=='2020-11-18': return 17
    elif x=='2020-11-19': return 18
    elif x=='2020-11-20': return 19
    elif x=='2020-11-21': return 20
    elif x=='2020-11-22': return 21
    elif x=='2020-11-23': return 22
    elif x=='2020-11-24': return 23
    elif x=='2020-11-25': return 24
    elif x=='2020-11-26': return 25
    elif x=='2020-11-27': return 26
    elif x=='2020-11-28': return 27
    elif x=='2020-11-29': return 28
    elif x=='2020-11-30': return 29
    else: return 0

In [ ]:
test_err['datetime']=test_err['datetime'].apply(test1)

In [ ]:
test_err.info()

In [ ]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((14999,42))
for person_idx, err in tqdm(id_error):
    test_x[person_idx - 30000,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
test = pd.DataFrame(data=test_x)

In [ ]:
id_error = test_err[['user_id','datetime']].values
test_x = np.zeros((14999,30))
for person_idx, err in tqdm(id_error):
    # person_idx - 10000 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - 30000,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
testt = pd.DataFrame(data=test_x)

In [ ]:
 test2=test.merge(testt,left_index=True, right_index=True )

In [ ]:
test_err_model=test_err[['user_id','model_nm']].drop_duplicates()
model_list=np.sort(test_err_model['model_nm'].unique())
Model = pd.DataFrame()

for i in tqdm(range(len(model_list))):
  model=np.zeros((14999))
  train_err_model1=test_err_model[test_err_model['model_nm']==model_list[i]].reset_index(drop=True)
  model[train_err_model1.user_id.unique()-30000]=1
  Model[model_list[i]]=model


In [ ]:
 test3=test2.merge(Model,left_index=True, right_index=True )

In [ ]:
Number_model = pd.DataFrame(columns=['user_id', 'Number_of_model'])
user_list=test_err.user_id.unique().tolist()
for i in tqdm(range(len(user_list))):
  count=len(test_err.loc[test_err.user_id==user_list[i]].model_nm.unique())
  Number_model=Number_model.append(pd.DataFrame([[user_list[i],count]],columns=['user_id','Number_of_model']),ignore_index=True)

In [ ]:
new_data={'user_id':'43262','Number_of_model':0}
Number_model=Number_model.append(new_data,ignore_index=True)

In [ ]:
 test4=test3.merge(Number_model,left_index=True, right_index=True )

In [ ]:
predictions = predict_model(final_model, data = test4)

In [ ]:
#pycaret에서는 score이 label을 맞출 확률이기때문에 output을 제출 양식에 맞게 바꿔줍니다
x = []
for i in range(len(predictions['Score'])):
  if predictions['Label'][i] =='1.0':
    x.append(predictions['Score'][i])
  else:
    x.append(1-predictions['Score'][i])


In [ ]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion['problem'] = x
sample_submssion.to_csv(CSV_PATH+"210203(3).csv", index = False)